In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scvelo as scv
import scanpy

In [ ]:
datdir = "../data"

## Gastrulation dataset accessed through scVelo

Load the gastrulation dataset provided by scVelo. It looks like this dataset doesn't contain all of the 116,312 cells that we are interested in. In particular, it appears to lack cells expressing Sox1 and Sox2, which are some of the marker proteins used by Sàez et al. We can see that there are no counts of Sox1 across all of the cells.

In [ ]:
adata = scv.datasets.gastrulation(f"{datdir}/Gastrulation/gastrulation.h5ad")
print(adata)

In [ ]:
all_genes = adata.var.index.str.lower()

# Find genes in the Sox 'family'
sox_genes = all_genes[all_genes.str.slice(0,3) == 'sox']

print("Total counts of Sox1:", adata[:,'Sox1'].X.sum())

Prepare the data, adopting the approach taken by S. Farrell [here](https://github.com/Spencerfar/LatentVelo/blob/main/paper_notebooks/Gastrulation.ipynb).

In [ ]:
# subsample data
np.random.seed(1)
# adata = adata[np.random.choice(adata.shape[0], size=30000, replace=False)]

# set up experimental time
adata.obs['exp_time'] = np.array([float(t[1:]) for t in adata.obs['stage']])
adata.obs['exp_time'] = adata.obs['exp_time']/adata.obs['exp_time'].max()

adata.obs['celltype_names'] = adata.obs['celltype'].copy().values
# scv.pp.filter_genes(adata, min_shared_counts=10)

# ltv.utils.anvi_clean_recipe(adata, batch_key='sequencing.batch', celltype_key='celltype', n_top_genes=None)


Take a look at the cell types present.

In [ ]:
list(np.unique(adata.obs['celltype_names']))

We can select a subset of these cells that we wish to keep. For now, keep all of them.

In [ ]:
# types_to_keep = [
#     'Anterior Primitive Streak',
#     'Epiblast', 
#     'Caudal epiblast', 
#     'Caudal neurectoderm', 
#     'Rostral neurectoderm', 
#     'Paraxial mesoderm',
#     'Caudal Mesoderm',
#     'Primitive Streak',
#     'NMP',
#     'Spinal cord'
# ]

types_to_keep = np.unique(adata.obs['celltype_names'])

subset = adata[np.isin(adata.obs['celltype_names'], types_to_keep)]

We are interested in only a subset of the genes.

In [ ]:
gene_list = [
    'T',  # Bra
    'Cdx2',
    'Sox1',
    'Sox2',
    'Sox3',
    'Tbx6',
    'Otx2'
]

present_list = []
for gname in gene_list:
    ispresent = gname in subset.var.index
    print(f"Gene {gname} in subset?: {ispresent}")
    present_list.append(ispresent)

In [ ]:
list(np.unique(subset.obs['celltype_names']))

### Plot all cells in the subset, coloring according to cell type.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ctypes = np.unique(subset.obs['celltype_names'])
for ctype in ctypes:
    bset = subset[subset.obs['celltype_names'] == ctype]
    ax.scatter(bset.obs['umapX'], bset.obs['umapY'], s=3, color=bset.obs['colour'], label=ctype)
ax.set_xlabel('UMAP $x$')
ax.set_ylabel('UMAP $y$')

lgnd = ax.legend()  
for i in range(len(ctypes)):
    lgnd.legendHandles[i]._sizes = [30]

### Plot gene expression for each gene of interest

In [ ]:
for i, gname in enumerate(gene_list):
    if not present_list[i]:
        print(f"Gene {gname} not present in subset. Skipping")
    else:
        fig, ax = plt.subplots(1, 1, figsize=(4, 4))
        
        for ctype in ctypes:
            bset = subset[subset.obs['celltype_names'] == ctype]
            ax.scatter(bset.obs['umapX'], bset.obs['umapY'], s=3, color=bset.obs['colour'], label=ctype,alpha=0.01)
        ax.set_xlabel('UMAP $x$')
        ax.set_ylabel('UMAP $y$')
            
        expr = subset[:,gname].X.toarray()
        screen = (expr > 0).flatten()
        expr = expr[screen]

        ax.scatter(subset.obs['umapX'][screen], subset.obs['umapY'][screen], s=1, c=expr, label=gname, alpha=0.2)
        ax.set_xlabel('UMAP $x$')
        ax.set_ylabel('UMAP $y$')
        ax.set_title(gname)

### Some silly quantitative analysis of the genes of interest

In [ ]:
for gname in gene_list:
    print(gname)
    tot_expr = np.sum(adata[:,gname].X, 1)
    tot_expr_sp = np.sum(adata[:,gname].layers['spliced'], 1)
    tot_expr_us = np.sum(adata[:,gname].layers['unspliced'], 1)
    screen = tot_expr > 0
    print("\tTotal Expr:\n\t", tot_expr[screen])
    print("\tTotal Expr (spliced):\n\t", tot_expr_sp[screen])
    print("\tTotal Expr (unspliced):\n\t", tot_expr_us[screen])

#     print(np.sum(adata[:,gname].layers['spliced'], 1).max())
#     print(np.sum(adata[:,gname].layers['unspliced'], 1).max())